In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import ta  # For technical indicators

df = pd.read_csv("C:/Users/NISHA SINGH/Downloads/STOCK MARKET/TATASTEEL1_cleaned.csv")
df.columns = df.columns.str.strip()
df = df[~df['Date'].astype(str).str.contains('Date', na=False)]
for col in ['Open', 'High', 'Low', 'Close']:
    df[col] = df[col].astype(str).str.replace(',', '', regex=True).astype(float)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
df.dropna(subset=['Date'], inplace=True)
df = df.set_index('Date', drop=False)

# Calculate Technical Indicators
df['MA15'] = df['Close'].rolling(window=15).mean()
df['MACD'] = ta.trend.MACD(close=df['Close']).macd()
df['K14'] = ta.momentum.stoch(high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3)
df['D3'] = df['K14'].rolling(window=3).mean()
df['RSI14'] = ta.momentum.RSIIndicator(close=df['Close'], window=14).rsi()
df['WR14'] = ta.momentum.WilliamsRIndicator(high=df['High'], low=df['Low'], close=df['Close'], lbp=14).williams_r()
df.dropna(inplace=True)

# Prepare Features and Target
features = df[['MA15', 'MACD', 'K14', 'D3', 'RSI14', 'WR14']]
target = df[['Close']]

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target)


def create_sequences(data, labels, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(labels[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(features_scaled, target_scaled, seq_length)

# Train-Test Split
split_ratio = 0.8
split_point = int(len(X) * split_ratio)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

#Define LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, X.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')


history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

pred_df = pd.DataFrame({'Actual': y_test_actual.flatten(), 'Predicted': y_pred.flatten()}, index=df.index[-len(y_test):])


y_full_pred = model.predict(X)
y_full_pred = scaler.inverse_transform(y_full_pred.reshape(-1, 1))
y_actual_full = scaler.inverse_transform(y.reshape(-1, 1))
full_pred_df = pd.DataFrame({'Actual': y_actual_full.flatten(), 'Predicted': y_full_pred.flatten()}, index=df.index[seq_length:])


fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=full_pred_df.index, y=full_pred_df['Actual'], mode='lines', name='Actual', line=dict(color='blue')))
fig1.add_trace(go.Scatter(x=full_pred_df.index, y=full_pred_df['Predicted'], mode='lines', name='Predicted', line=dict(color='red')))
fig1.update_layout(title="Actual vs Predicted Closing Price (Entire Dataset)", xaxis_title="Date", yaxis_title="Price", hovermode='x unified')


fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=pred_df.index, y=pred_df['Actual'], mode='lines', name='Actual', line=dict(color='blue')))
fig2.add_trace(go.Scatter(x=pred_df.index, y=pred_df['Predicted'], mode='lines', name='Predicted', line=dict(color='red')))
fig2.update_layout(title="Actual vs Predicted Closing Price (Last 5 Years)", xaxis_title="Date", yaxis_title="Price", hovermode='x unified')


fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=full_pred_df.index[-60:], y=full_pred_df['Actual'][-60:], mode='lines', name='Actual', line=dict(color='blue')))
fig3.add_trace(go.Scatter(x=full_pred_df.index[-60:], y=full_pred_df['Predicted'][-60:], mode='lines', name='Predicted', line=dict(color='red')))
fig3.update_layout(title="Actual vs Predicted Closing Price (Last 2 Months)", xaxis_title="Date", yaxis_title="Price", hovermode='x unified')

fig1.show()
fig2.show()
fig3.show()


In [ ]:
full_pred_df['Predicted_Change'] = full_pred_df['Predicted'].diff()

# Set threshold for "NO TREND"
threshold = 0.1

# Generate Trend based on the change
def get_trend(change):
    if change > threshold:
        return 'UP'
    elif change < -threshold:
        return 'DOWN'
    else:
        return 'NO TREND'

full_pred_df['Trend'] = full_pred_df['Predicted_Change'].apply(get_trend)


pd.set_option('display.max_rows', 250)  
print(full_pred_df[['Predicted', 'Predicted_Change', 'Trend']].tail(50))



In [ ]:
full_pred_df['Predicted_Change'] = full_pred_df['Predicted'].diff()
threshold = 0.5
def get_trend(change):
    if change > threshold:
        return 'UP'
    elif change < -threshold:
        return 'DOWN'
    else:
        return 'NO TREND'

full_pred_df['Trend'] = full_pred_df['Predicted_Change'].apply(get_trend)

x_min = full_pred_df['Predicted_Change'].min()
x_max = full_pred_df['Predicted_Change'].max()
full_pred_df['Signal'] = (full_pred_df['Predicted_Change'] - x_min) / (x_max - x_min)

pd.set_option('display.max_rows', 250)

print(full_pred_df[['Predicted', 'Predicted_Change', 'Trend', 'Signal']].tail(20))


In [ ]:
def trading_decision(signal):
    if signal > 0.7:
        return 'BUY'
    elif signal < 0.3:
        return 'SELL'
    else:
        return ' '  

full_pred_df['Trading_Decision'] = full_pred_df['Signal'].apply(trading_decision)
pd.set_option('display.max_rows', 250)
print(full_pred_df[['Predicted', 'Predicted_Change', 'Trend', 'Trading_Decision']].tail(50))


In [ ]:
buy_price = None
profits = []

for index, row in full_pred_df.iterrows():
    if row['Trading_Decision'] == 'BUY' and buy_price is None:
        buy_price = row['Predicted']
    elif row['Trading_Decision'] == 'SELL' and buy_price is not None:
        sell_price = row['Predicted']
        profit_percent = ((sell_price - buy_price) / buy_price) * 100
        profits.append(profit_percent)
        buy_price = None  

total_profit_percent = sum(profits)

print(f"Total Profit Percentage: {total_profit_percent:.2f}%")


In [ ]:
import plotly.graph_objs as go
import pandas as pd

plt.figure(figsize=(10, 6))

last_200 = full_pred_df.tail(300)

# Create the main price line
price_trace = go.Scatter(
    x=last_200.index,
    y=last_200['Actual'],
    mode='lines',
    name='Actual Closing Price',
    line=dict(color='blue')
)

buy_trace = go.Scatter(
    x=last_200[last_200['Trading_Decision'] == 'BUY'].index,
    y=last_200[last_200['Trading_Decision'] == 'BUY']['Actual'],
    mode='markers',
    name='BUY Signal',
    marker=dict(symbol='triangle-up', color='green', size=12)
)

sell_trace = go.Scatter(
    x=last_200[last_200['Trading_Decision'] == 'SELL'].index,
    y=last_200[last_200['Trading_Decision'] == 'SELL']['Actual'],
    mode='markers',
    name='SELL Signal',
    marker=dict(symbol='triangle-down', color='red', size=12)
)

layout = go.Layout(
    title='Actual Closing Price with Buy/Sell Signals (Interactive)',
    xaxis=dict(title='Date', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    yaxis=dict(title='Price', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    hovermode='x unified'
)

fig = go.Figure(data=[price_trace, buy_trace, sell_trace], layout=layout)

fig.show()

In [ ]:
last_signal = full_pred_df['Signal'].tail(200).values

plt.figure(figsize=(10, 6))

avg_signal = np.mean(last_signal)
plt.figure(figsize=(10, 6))
plt.plot(last_signal, color='lightgreen', label='trading signal')
plt.axhline(y=avg_signal, color='black', linestyle='--', linewidth=2, label='avg of trading signal')

plt.xlabel('Days')
plt.ylabel('Trading signal generated from CEFLANN')
plt.title('Output of CEFLANN for the dataset')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

full_pred_df['Actual_Change'] = full_pred_df['Actual'].diff()
full_pred_df['Actual_Trend'] = full_pred_df['Actual_Change'].apply(get_trend)

trend_df = full_pred_df.dropna(subset=['Trend', 'Actual_Trend'])

cm = confusion_matrix(trend_df['Actual_Trend'], trend_df['Trend'], labels=['UP', 'DOWN', 'NO TREND'])
labels = ['UP', 'DOWN', 'NO TREND']
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title("Confusion Matrix: Actual vs Predicted Trends")
plt.xlabel("Predicted Trend")
plt.ylabel("Actual Trend")
plt.show()

print("\nClassification Report:\n")
print(classification_report(trend_df['Actual_Trend'], trend_df['Trend'], target_names=labels))
from sklearn.metrics import accuracy_score

# Calculate Accuracy
accuracy = accuracy_score(trend_df['Actual_Trend'], trend_df['Trend'])
print(f"\nAccuracy of Trend Prediction: {accuracy * 100:.2f}%")



In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

features = ['MA15', 'MACD', 'K14', 'D3', 'RSI14', 'WR14']
target = 'Close' 
scaler = MinMaxScaler()
X = scaler.fit_transform(df[features])
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

svr_model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.01)
svr_model.fit(X_train, y_train)
predictions = svr_model.predict(X_test)

svm_pred_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predictions
})
svm_pred_df['Predicted_Change'] = svm_pred_df['Predicted'].diff()

threshold = 0.5 
def get_decision(change):
    if change > threshold:
        return 'BUY'
    elif change < -threshold:
        return 'SELL'
    else:
        return 'HOLD'

svm_pred_df['Trading_Decision'] = svm_pred_df['Predicted_Change'].apply(get_decision)

buy_price = None
profits = []

for index, row in svm_pred_df.iterrows():
    if row['Trading_Decision'] == 'BUY' and buy_price is None:
        buy_price = row['Predicted']
    elif row['Trading_Decision'] == 'SELL' and buy_price is not None:
        sell_price = row['Predicted']
        profit_percent = ((sell_price - buy_price) / buy_price) * 100
        profits.append(profit_percent)
        buy_price = None  # Reset after selling


total_profit_percent = sum(profits)
print(f"Total Profit Percentage using SVM: {total_profit_percent:.2f}%")


In [ ]:
import plotly.graph_objs as go
import pandas as pd

plt.figure(figsize=(10, 6))

full_pred_df = full_pred_df.iloc[-len(y_test):].copy()

full_pred_df['Date'] = df.iloc[-len(y_test):]['Date'].values

last_200 = full_pred_df.tail(200).set_index('Date')  # Set Date as index

price_trace = go.Scatter(
    x=last_200.index,
    y=last_200['Actual'],
    mode='lines',
    name='Actual Closing Price',
    line=dict(color='blue')
)

buy_trace = go.Scatter(
    x=last_200[last_200['Trading_Decision'] == 'BUY'].index,
    y=last_200[last_200['Trading_Decision'] == 'BUY']['Actual'],
    mode='markers',
    name='BUY Signal',
    marker=dict(symbol='triangle-up', color='green', size=12)
)

sell_trace = go.Scatter(
    x=last_200[last_200['Trading_Decision'] == 'SELL'].index,
    y=last_200[last_200['Trading_Decision'] == 'SELL']['Actual'],
    mode='markers',
    name='SELL Signal',
    marker=dict(symbol='triangle-down', color='red', size=12)
)

layout = go.Layout(
    title='Actual Closing Price with Buy/Sell Signals (SVM)',
    xaxis=dict(title='Date', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    yaxis=dict(title='Price', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    hovermode='x unified'
)

fig = go.Figure(data=[price_trace, buy_trace, sell_trace], layout=layout)
fig.show()

import plotly.graph_objs as go

# Prepare comparison data
compare_df = full_pred_df.tail(200).copy()  # Use last 200 for comparison
compare_df.set_index('Date', inplace=True)

# Trace for actual prices
actual_trace = go.Scatter(
    x=compare_df.index,
    y=compare_df['Actual'],
    mode='lines',
    name='Actual Closing Price',
    line=dict(color='blue')
)

# Trace for predicted prices
predicted_trace = go.Scatter(
    x=compare_df.index,
    y=compare_df['Predicted'],
    mode='lines',
    name='Predicted Closing Price (SVM)',
    line=dict(color='red')
)

# Layout for second plot
layout2 = go.Layout(
    title='Actual vs Predicted Closing Price (SVM)',
    xaxis=dict(title='Date', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    yaxis=dict(title='Price', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    hovermode='x unified'
)

# Plot second graph
fig2 = go.Figure(data=[actual_trace, predicted_trace], layout=layout2)
fig2.show()



In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df.dropna(inplace=True)

scaler = MinMaxScaler()
X = scaler.fit_transform(df[features])
y = df['Target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = GaussianNB()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
nb_pred_df = pd.DataFrame({
    'Actual_Close': df['Close'].iloc[-len(predictions):].values,
    'Predicted_Signal': predictions
})
buy_price = None
profits = []

for index, row in nb_pred_df.iterrows():
    if row['Predicted_Signal'] == 1 and buy_price is None:
        buy_price = row['Actual_Close']
    elif row['Predicted_Signal'] == 0 and buy_price is not None:
        sell_price = row['Actual_Close']
        profit_percent = ((sell_price - buy_price) / buy_price) * 100
        profits.append(profit_percent)
        buy_price = None
total_profit_percent = sum(profits)
print(f"Total Profit Percentage using Naive Bayes: {total_profit_percent:.2f}%")


In [ ]:
import plotly.graph_objs as go
import pandas as pd

# Prepare data
nb_pred_df['Date'] = df.iloc[-len(predictions):]['Date'].values
nb_pred_df['Trading_Decision'] = nb_pred_df['Predicted_Signal'].apply(lambda x: 'BUY' if x == 1 else 'SELL')
last_200 = nb_pred_df.tail(200).copy()
last_200.set_index('Date', inplace=True)

# Plot actual price
price_trace = go.Scatter(
    x=last_200.index,
    y=last_200['Actual_Close'],
    mode='lines',
    name='Actual Closing Price',
    line=dict(color='blue')
)

# Plot BUY signals
buy_trace = go.Scatter(
    x=last_200[last_200['Trading_Decision'] == 'BUY'].index,
    y=last_200[last_200['Trading_Decision'] == 'BUY']['Actual_Close'],
    mode='markers',
    name='BUY Signal',
    marker=dict(symbol='triangle-up', color='green', size=12)
)

# Plot SELL signals
sell_trace = go.Scatter(
    x=last_200[last_200['Trading_Decision'] == 'SELL'].index,
    y=last_200[last_200['Trading_Decision'] == 'SELL']['Actual_Close'],
    mode='markers',
    name='SELL Signal',
    marker=dict(symbol='triangle-down', color='red', size=12)
)

# Layout
layout1 = go.Layout(
    title='Actual Closing Price with Buy/Sell Signals (Naive Bayes)',
    xaxis=dict(title='Date', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    yaxis=dict(title='Price', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    hovermode='x unified'
)

# Show plot
fig1 = go.Figure(data=[price_trace, buy_trace, sell_trace], layout=layout1)
fig1.show()


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df.dropna(inplace=True)
scaler = MinMaxScaler()
X = scaler.fit_transform(df[features])
y = df['Target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

predictions = knn.predict(X_test)

knn_pred_df = pd.DataFrame({
    'Actual_Close': df['Close'].iloc[-len(predictions):].values,
    'Predicted_Signal': predictions
})

buy_price = None
profits = []

for index, row in knn_pred_df.iterrows():
    if row['Predicted_Signal'] == 1 and buy_price is None:
        buy_price = row['Actual_Close']
    elif row['Predicted_Signal'] == 0 and buy_price is not None:
        sell_price = row['Actual_Close']
        profit_percent = ((sell_price - buy_price) / buy_price) * 100
        profits.append(profit_percent)
        buy_price = None

total_profit_percent = sum(profits)
print(f"Total Profit Percentage using KNN: {total_profit_percent:.2f}%")


In [ ]:
import plotly.graph_objs as go
import pandas as pd

plt.figure(figsize=(10, 6))
knn_pred_df = knn_pred_df.iloc[-len(y_test):].copy()
knn_pred_df['Date'] = df.iloc[-len(y_test):]['Date'].values

knn_pred_df_last_200 = knn_pred_df.tail(200).set_index('Date')  # Set Date as index

buy_signals = knn_pred_df_last_200[knn_pred_df_last_200['Predicted_Signal'] == 1]
sell_signals = knn_pred_df_last_200[knn_pred_df_last_200['Predicted_Signal'] == 0]

trace_price = go.Scatter(
    x=knn_pred_df_last_200.index,
    y=knn_pred_df_last_200['Actual_Close'],
    mode='lines',
    name='Actual Close Price',
    line=dict(color='black')
)

trace_buy = go.Scatter(
    x=buy_signals.index,
    y=buy_signals['Actual_Close'],
    mode='markers',
    name='BUY Signal',
    marker=dict(symbol='triangle-up', color='green', size=12)
)

trace_sell = go.Scatter(
    x=sell_signals.index,
    y=sell_signals['Actual_Close'],
    mode='markers',
    name='SELL Signal',
    marker=dict(symbol='triangle-down', color='red', size=12)
)

layout = go.Layout(
    title='KNN Buy/Sell Signals on Stock Prices (Interactive)',
    xaxis=dict(title='Date', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    yaxis=dict(title='Price', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    hovermode='x unified'
)

fig = go.Figure(data=[trace_price, trace_buy, trace_sell], layout=layout)
fig.show()


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df.dropna(inplace=True)

scaler = MinMaxScaler()
X = scaler.fit_transform(df[features])
y = df['Target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)

predictions = dt_model.predict(X_test)

dt_pred_df = pd.DataFrame({
    'Actual_Close': df['Close'].iloc[-len(predictions):].values,
    'Predicted_Signal': predictions
})

buy_price = None
profits = []

for index, row in dt_pred_df.iterrows():
    if row['Predicted_Signal'] == 1 and buy_price is None:
        buy_price = row['Actual_Close']
    elif row['Predicted_Signal'] == 0 and buy_price is not None:
        sell_price = row['Actual_Close']
        profit_percent = ((sell_price - buy_price) / buy_price) * 100
        profits.append(profit_percent)
        buy_price = None

total_profit_percent = sum(profits)
print(f"Total Profit Percentage using Decision Tree: {total_profit_percent:.2f}%")


In [ ]:
import plotly.graph_objs as go
import pandas as pd

plt.figure(figsize=(10, 6))

dt_pred_df = dt_pred_df.iloc[-len(y_test):].copy()

dt_pred_df['Date'] = df.iloc[-len(y_test):]['Date'].values

dt_pred_df_last_200 = dt_pred_df.tail(200).set_index('Date')  # Set Date as index

buy_signals = dt_pred_df_last_200[dt_pred_df_last_200['Predicted_Signal'] == 1]
sell_signals = dt_pred_df_last_200[dt_pred_df_last_200['Predicted_Signal'] == 0]

trace_price = go.Scatter(
    x=dt_pred_df_last_200.index,
    y=dt_pred_df_last_200['Actual_Close'],
    mode='lines',
    name='Actual Close Price',
    line=dict(color='black')
)

trace_buy = go.Scatter(
    x=buy_signals.index,
    y=buy_signals['Actual_Close'],
    mode='markers',
    name='BUY Signal',
    marker=dict(symbol='triangle-up', color='green', size=12)
)

trace_sell = go.Scatter(
    x=sell_signals.index,
    y=sell_signals['Actual_Close'],
    mode='markers',
    name='SELL Signal',
    marker=dict(symbol='triangle-down', color='red', size=12)
)

layout = go.Layout(
    title='Decision Tree Buy/Sell Signals on Stock Prices (Interactive)',
    xaxis=dict(title='Date', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    yaxis=dict(title='Price', showspikes=True, spikemode='across', spikesnap='cursor', showline=True),
    hovermode='x unified'
)

fig = go.Figure(data=[trace_price, trace_buy, trace_sell], layout=layout)
fig.show()
